### Loading packages

In [17]:
import os
import sys
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sqlite3
from sqlite3 import Error

### Loading [Data from Kaggle competition](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions)

In [18]:
path = './data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

./data/RAW_recipes.csv
./data/interactions_test.csv
./data/interactions_validation.csv
./data/PP_recipes.csv
./data/.~lock.PP_recipes.csv#
./data/PP_users.csv
./data/interactions_train.csv
./data/RAW_interactions.csv


In [19]:
df_recipes_raw = pd.read_csv('./data/RAW_recipes.csv')
df_recipes_cod = pd.read_csv('./data/PP_recipes.csv')

### Examining Data

In [20]:
df_recipes_raw.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [21]:
df_recipes_cod.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [22]:
df_recipes = pd.merge(df_recipes_cod, df_recipes_raw, on='id')
df_recipes.set_index(["id"], drop=True, inplace=True)
#df_recipes.set_index(["i"], drop=True, inplace=True)
#df_recipes.drop(['i'], axis=1, inplace=True)

In [23]:
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178265 entries, 424415 to 263840
Data columns (total 18 columns):
i                    178265 non-null int64
name_tokens          178265 non-null object
ingredient_tokens    178265 non-null object
steps_tokens         178265 non-null object
techniques           178265 non-null object
calorie_level        178265 non-null int64
ingredient_ids       178265 non-null object
name                 178265 non-null object
minutes              178265 non-null int64
contributor_id       178265 non-null int64
submitted            178265 non-null object
tags                 178265 non-null object
nutrition            178265 non-null object
n_steps              178265 non-null int64
steps                178265 non-null object
description          174311 non-null object
ingredients          178265 non-null object
n_ingredients        178265 non-null int64
dtypes: int64(6), object(12)
memory usage: 25.8+ MB


In [24]:
df_recipes.iloc[1]["ingredients"]

"['flour', 'oats', 'brown sugar', 'pecans', 'butter', 'eggs', 'solid pack pumpkin', 'sweetened condensed milk', 'cinnamon', 'ginger', 'nutmeg', 'salt']"

In [25]:
df_recipes.iloc[1]["ingredient_ids"]

'[2683, 4969, 800, 5298, 840, 2499, 6632, 7022, 1511, 3248, 4964, 6270]'

#### Examining the "Tag" field

In [26]:
df_recipes.tags.head()

id
424415    ['weeknight', 'time-to-make', 'course', 'main-...
146223    ['60-minutes-or-less', 'time-to-make', 'course...
312329    ['30-minutes-or-less', 'time-to-make', 'course...
74301     ['15-minutes-or-less', 'time-to-make', 'course...
76272     ['30-minutes-or-less', 'time-to-make', 'course...
Name: tags, dtype: object

In [27]:
tags = set([])
for taglist in df_recipes.tags:
    taglist = eval(taglist)
    for tag in taglist:
        tags.add(tag)

In [28]:
len(tags)

530

#### Examining the "Ingredients" field

In [29]:
ingredients = set([])
for ingredient_list in df_recipes.ingredients:
    ingredient_list = eval(ingredient_list)
    for ingredient in ingredient_list:
        ingredients.add(ingredient)

In [30]:
len(ingredients)

13627

### Exporting an abridged dataframe for similarity calculations

In [32]:
df_recipes.rename(columns={"i": "old_id",}, inplace=True)
df_recipes.columns

Index(['old_id', 'name_tokens', 'ingredient_tokens', 'steps_tokens',
       'techniques', 'calorie_level', 'ingredient_ids', 'name', 'minutes',
       'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps',
       'description', 'ingredients', 'n_ingredients'],
      dtype='object')

In [33]:
useful = ['name_tokens', 
          'ingredient_tokens', 
          'steps_tokens', 
          'techniques',
          'calorie_level',
          'ingredient_ids',
          'name', 
          'minutes',
          'tags', 
          'nutrition', 
          'steps', 
          'ingredients', 
          'old_id', 
          ]
df_recipes[useful].head()

,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids,name,minutes,tags,nutrition,steps,ingredients,old_id
id,,,,,,,,,,,,,
424415,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]",aromatic basmati rice rice cooker,61,"['weeknight', 'time-to-make', 'course', 'main-...","[228.2, 2.0, 2.0, 8.0, 9.0, 1.0, 15.0]","['rinse the rice in a fine strainer , then dra...","['basmati rice', 'water', 'salt', 'cinnamon st...",23
146223,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,...",pumpkin pie a la easy,55,"['60-minutes-or-less', 'time-to-make', 'course...","[249.4, 16.0, 92.0, 8.0, 11.0, 27.0, 11.0]","['preheat oven to 350', 'combine flour , oats ...","['flour', 'oats', 'brown sugar', 'pecans', 'bu...",96900
312329,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696...",cheesy tomato soup with potatoes,25,"['30-minutes-or-less', 'time-to-make', 'course...","[351.3, 34.0, 15.0, 50.0, 25.0, 70.0, 8.0]","['pour the broth & water into a large pot', 'a...","['chicken broth', 'water', 'salt', 'black pepp...",120056
74301,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]",mini tacos,15,"['15-minutes-or-less', 'time-to-make', 'course...","[79.7, 5.0, 2.0, 11.0, 11.0, 7.0, 2.0]","['cook hamburger until browned', 'drain the fa...","['wonton wrappers', 'hamburger', 'taco seasoni...",168258
76272,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]",rosemary s hanky panky s,20,"['30-minutes-or-less', 'time-to-make', 'course...","[240.7, 29.0, 9.0, 28.0, 27.0, 42.0, 0.0]","['fry ground beef and sausage until browned', ...","['ground beef', 'ground sausage', 'velveeta ch...",109030


In [34]:
df_recipes[useful].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178265 entries, 424415 to 263840
Data columns (total 13 columns):
name_tokens          178265 non-null object
ingredient_tokens    178265 non-null object
steps_tokens         178265 non-null object
techniques           178265 non-null object
calorie_level        178265 non-null int64
ingredient_ids       178265 non-null object
name                 178265 non-null object
minutes              178265 non-null int64
tags                 178265 non-null object
nutrition            178265 non-null object
steps                178265 non-null object
ingredients          178265 non-null object
old_id               178265 non-null int64
dtypes: int64(3), object(10)
memory usage: 19.0+ MB


In [35]:
df_recipes.name_tokens = df_recipes.name_tokens.apply(eval)
df_recipes.ingredient_tokens = df_recipes.ingredient_tokens.apply(eval)
df_recipes.steps_tokens = df_recipes.steps_tokens.apply(eval)
df_recipes.techniques = df_recipes.techniques.apply(eval)
df_recipes.ingredient_ids = df_recipes.ingredient_ids.apply(eval)
df_recipes.tags = df_recipes.tags.apply(eval)
df_recipes.ingredients = df_recipes.ingredients.apply(eval)
df_recipes.nutrition = df_recipes.nutrition.apply(eval)
df_recipes.steps = df_recipes.steps.apply(eval)

In [36]:
df_recipes.to_pickle("basic_features.pkl.bz2")

### Building a database from the data (TBFinished)

In [62]:
db_file = 'recipes.sqlite'

def create_connection(db_file):
    """ 
    create a database connection to the SQLite database
    specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


def create_table(conn, create_table_sql):
    """ 
    create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [71]:
sql_create_ingredients_table = r""" CREATE TABLE IF NOT EXISTS ingredients (
                               id integer PRIMARY KEY,
                               ingredient_name text NOT NULL
                               ); """
 
sql_create_tags_table = r"""CREATE TABLE IF NOT EXISTS tags (
                        id integer PRIMARY KEY,
                        tag_name text NOT NULL
                        );"""
 
sql_create_recipes_table = r"""CREATE TABLE IF NOT EXISTS recipes (
                           id integer PRIMARY KEY,
                           recipe_name text NOT NULL
                           );"""   

sql_create_recipes_tags = r"""CREATE TABLE IF NOT EXISTS recipes_tags (
                           id integer PRIMARY KEY,
                           recipe_id integer NOT NULL,
                           tag_id integer NOT NULL,
                           FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                           FOREIGN KEY (tag_id) REFERENCES tags (id)
                           );""" 

sql_create_recipes_ingredients = r"""CREATE TABLE IF NOT EXISTS recipes_ingredients (
                                 id integer PRIMARY KEY,
                                 recipe_id integer NOT NULL,
                                 ingredient_id integer NOT NULL,
                                 FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                                 FOREIGN KEY (ingredient_id) REFERENCES ingredients (id)
                                 );""" 

# create a database connection
conn = create_connection(db_file)

# create tables
if conn is not None:
    create_table(conn, sql_create_ingredients_table)
    create_table(conn, sql_create_tags_table)
    create_table(conn, sql_create_recipes_table)
    create_table(conn, sql_create_recipes_tags)
    create_table(conn, sql_create_recipes_ingredients)
else:
    print("Error! cannot create the database connection.")

In [72]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, ingredient in enumerate(ingredients):
    cur.execute('INSERT INTO ingredients VALUES(?,?)',(idx, ingredient))

In [73]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, tag in enumerate(tags):
    cur.execute('INSERT INTO tags VALUES(?,?)',(idx, tag))

In [76]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, name in enumerate(df_recipes.name):
    try:
        cur.execute('INSERT INTO recipes VALUES(?,?)',(idx, name))
    except:
        print(name)

nan


In [77]:
cur.execute("select * from tags limit 10;")
for result in cur.fetchall():
    print(result)